# Node

In [16]:
!pip list

Package                    Version             Location
-------------------------- ------------------- ------------------------------------
aioice                     0.6.18
aiortc                     0.9.28
aiounittest                1.4.0
alembic                    1.4.2
ansiwrap                   0.8.4
appdirs                    1.4.4
async-generator            1.10
attrs                      19.3.0
aurora-data-api            0.2.0
av                         8.0.2
backcall                   0.2.0
bcrypt                     3.2.0
black                      20.8b1
bleach                     3.1.5
boto3                      1.15.5
botocore                   1.18.5
certifi                    2020.6.20
cffi                       1.14.1
cfgv                       3.2.0
chardet                    3.0.4
click                      7.1.2
colorama                   0.4.3
coverage                   4.5.4
crc32c                     2.0.1
cryptography               3.0
decorator                  

In [5]:
import time
import torch as th
th.set_num_threads(1)
import syft as sy
from syft.grid.clients.data_centric_fl_client import DataCentricFLClient
hook = sy.TorchHook(th)
alice = DataCentricFLClient(hook, "ws://localhost:7600")
bob = DataCentricFLClient(hook, "ws://localhost:7601")
my_grid = sy.PrivateGridNetwork(alice,bob)
payload = 1_000 # in Mo
t = th.rand(1, int(payload * 10**6 / 4)) # 4 because torch float32 are encded over 4 bytes
timer = time.time()
# t.send(alice)
delay = time.time() - timer
print('bandwidth', payload/delay)

bandwidth 25.642299934877315


In [17]:
import argparse
import ast
import threading
import time

import pyarrow
import pyarrow.flight
import pyarrow.csv as csv

class FlightServer(pyarrow.flight.FlightServerBase):
    def __init__(self, host="localhost", location=None,
                 tls_certificates=None, verify_client=False,
                 root_certificates=None, auth_handler=None):
        super(FlightServer, self).__init__(
            location, auth_handler, tls_certificates, verify_client,
            root_certificates)
        self.flights = {}
        self.host = host
        self.tls_certificates = tls_certificates

    @classmethod
    def descriptor_to_key(self, descriptor):
        return (descriptor.descriptor_type.value, descriptor.command,
                tuple(descriptor.path or tuple()))

    def _make_flight_info(self, key, descriptor, table):
        if self.tls_certificates:
            location = pyarrow.flight.Location.for_grpc_tls(
                self.host, self.port)
        else:
            location = pyarrow.flight.Location.for_grpc_tcp(
                self.host, self.port)
        endpoints = [pyarrow.flight.FlightEndpoint(repr(key), [location]), ]

        mock_sink = pyarrow.MockOutputStream()
        stream_writer = pyarrow.RecordBatchStreamWriter(
            mock_sink, table.schema)
        stream_writer.write_table(table)
        stream_writer.close()
        data_size = mock_sink.size()

        return pyarrow.flight.FlightInfo(table.schema,
                                         descriptor, endpoints,
                                         table.num_rows, data_size)

    def list_flights(self, context, criteria):
        for key, table in self.flights.items():
            if key[1] is not None:
                descriptor = \
                    pyarrow.flight.FlightDescriptor.for_command(key[1])
            else:
                descriptor = pyarrow.flight.FlightDescriptor.for_path(*key[2])

            yield self._make_flight_info(key, descriptor, table)

    def get_flight_info(self, context, descriptor):
        key = FlightServer.descriptor_to_key(descriptor)
        if key in self.flights:
            table = self.flights[key]
            return self._make_flight_info(key, descriptor, table)
        raise KeyError('Flight not found.')

    def do_put(self, context, descriptor, reader, writer):
        key = FlightServer.descriptor_to_key(descriptor)
        t = time.time()
        print(f"Got a new key: {key}")

        table = reader.read_all()
        # self.flights[key] = reader.read_all()
        dl_time = time.time() - t
        print("Read all:")
        # print(self.flights[key])
        print(table.to_pandas())
        print(table.shape)
        print(f"Time: {dl_time}")


    def do_get(self, context, ticket):
        key = ast.literal_eval(ticket.ticket.decode())
        if key not in self.flights:
            return None
        return pyarrow.flight.RecordBatchStream(self.flights[key])

    def list_actions(self, context):
        return [
            ("clear", "Clear the stored flights."),
            ("shutdown", "Shut down this server."),
        ]

    def do_action(self, context, action):
        if action.type == "clear":
            raise NotImplementedError(
                "{} is not implemented.".format(action.type))
        elif action.type == "healthcheck":
            pass
        elif action.type == "shutdown":
            yield pyarrow.flight.Result(pyarrow.py_buffer(b'Shutdown!'))
            # Shut down on background thread to avoid blocking current
            # request
            threading.Thread(target=self._shutdown).start()
        else:
            raise KeyError("Unknown action {!r}".format(action.type))

    def _shutdown(self):
        """Shut down after a delay."""
        print("Server is shutting down...")
        time.sleep(2)
        self.shutdown()


In [ ]:
tls_certificates = []
scheme = "grpc+tcp"
host="localhost"
port="7604"

location = "{}://{}:{}".format(scheme, host, port)

server = FlightServer(host, location)

print("Serving on", location)
server.serve()